In [6]:
## 实验输入输出
import pandas as pd
eval_df = pd.read_json("../resources/data/cleaned/FinCUGE.jsonl", lines=True)
eval_df = eval_df[(eval_df['task']== 'FINNA') & (eval_df['split'] == 'eval')].reset_index()
pred_path =  "results/新闻摘要-qwen2_5_3B-sft.xlsx"

## 启动被测模型的vllm服务

vllm serve resources/open_models/Qwen2.5-3B-Instruct --trust-remote-code \
--served-model-name test \
--max-model-len 3072 --max-num-seqs 16 \
--tensor-parallel-size 4 --pipeline-parallel-size 2 --gpu-memory-utilization 0.15 \
--quantization fp8 \
--enable-lora \
--lora-modules test-lora=resources/ckpts/qwen2.5-3B-Instruct/lora_adapter \
--port 12234


## 启动评审模型的vllm服务

vllm serve resources/open_models/Qwen2.5-14B-Instruct  --trust-remote-code \
--served-model-name judger \
--max-model-len 10000 --max-num-seqs 16 \
--tensor-parallel-size 4 --pipeline-parallel-size 2 --gpu-memory-utilization 0.4 --enforce-eager \
--quantization fp8 --kv-cache-dtype fp8 \
--port 12235

## 预测

In [9]:
from openai import OpenAI
import pandas as pd
import os

test_model = OpenAI(base_url="http://localhost:12234/v1",api_key="empty")
test_model_name = "test-lora" ## 注意和要测的一致！特别是vllm带lora时！

## warm up
chat_completion = test_model.chat.completions.create(
    model=test_model_name,
    temperature=0.1, top_p=0.9, 
    messages=[{"role": "user","content": "字节跳动是什么时候成立的？"}],
)
print(chat_completion.choices[0].message.content) 

字节跳动成立于2012年。


In [11]:
if os.path.exists(pred_path):
    print("结果文件已经存在，跳过预测。")
else:
    print("预测...")
    for i, row in eval_df.iterrows():
        input_msg = [dict(role="system",content=row['instruction']),dict(role="user",content=row['input'])]
        chat_completion = test_model.chat.completions.create(model=test_model_name,temperature=0.1, top_p=1, messages=input_msg)
        pred = chat_completion.choices[0].message.content
        eval_df.loc[i,"prediction"] = pred
        print(f"{i} pred: {pred}")
    eval_df.to_excel(pred_path)

预测...
0 pred: 迈克生物定增结果出炉 高毅资产获配近2.7亿元
1 pred: 美国财政部宣布对4名乌克兰官员实施制裁
2 pred: 福建：2022年力争国家高新技术企业突破9000家
3 pred: 中金：关注年报有望超预期或者业绩环比逆势改善的公司
4 pred: 人民日报：让手机应用“长辈版”更贴心
5 pred: 收评：三大指数午后持续走弱沪指跌1% 白酒股走弱
6 pred: 英国卫生大臣：奥密克戎毒株防控工作已取得转机
7 pred: 美国银行：美联储将在3月加息，6月宣布缩表
8 pred: 31省份2021年居民人均可支配收入：上海北京超7万元
9 pred: 收评：三大指数午后震荡走高沪指涨0.8% 数字货币概念股领涨
10 pred: 新西兰外交部：汤加已建立起一个临时通信系统
11 pred: 2022年春运今天开启 除夕火车票正式开售
12 pred: 美国新冠肺炎日均新增确诊病例达777453例
13 pred: 10年间中国民营企业数量翻两番
14 pred: Peloton被剔除纳斯达克100指数，股价12个月跌幅达80%
15 pred: 韩国单日新增境外输入病例连续两天刷新最高值
16 pred: 花旗：对股票“略微”超配，建议逢低吸纳
17 pred: 美国国债期货下跌，逼近期权交易员的关键水平
18 pred: 招商中证白酒指数基金：四季度规模缩水152亿元
19 pred: 上海钢联：锂电产业链产品价格继续上涨
20 pred: 日本央行将上调通胀预期？市场预期日本央行将维持其超宽松货币政策不变
21 pred: 德国下调2022年经济增长预期至3.6%
22 pred: 德国新政府首次闭门内阁会议：应对气候变化是G7轮值主席国重要任务
23 pred: 美国累计新冠确诊病例数达6383万
24 pred: 东珠生态：与永平县政府签署项目投资框架协议
25 pred: 哈萨克斯坦副总理兼外长：哈方将制定新的投资政策概念
26 pred: 西安：2月10日起逐步恢复省际客运班线56条
27 pred: 立陶宛将派出迄今为止规模最大的代表团参加北京冬奥会
28 pred: 工信部：来电显示费将在3-5年内逐步取消
29 pred: 商务部：中俄货物贸易额历史上首次突破1400亿美元大关
30 pred: 董承非卸任兴

## 评估

In [4]:
# LLM评估综述 https://blog.csdn.net/m0_59164304/article/details/142148468
# Deep-Eval https://blog.csdn.net/lovechris00/article/details/143783278
import pandas as pd
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric,GEval
from deepeval.test_case import LLMTestCase
from langchain.chat_models import ChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM
from deepeval.dataset import EvaluationDataset
from deepeval.test_case import LLMTestCaseParams

class LLM(DeepEvalBaseLLM):
    def __init__(
        self,
        base_url,
        model_name,
        openai_api_key,
    ):
        self.model = ChatOpenAI(base_url=base_url,model_name=model_name,openai_api_key=openai_api_key)

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom vllm Server Model"

judger = LLM(base_url="http://localhost:12235/v1",model_name='judger',openai_api_key="empty")

correctness_metric = GEval(
    name="Correctness",
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradict any facts in 'expected output'. If there is a contradiction, the output is incorrect.",
        "Check if the 'actual output' omits any important details from the 'expected output'. If details are omitted, the output is less accurate.",
        "Evaluate the length of the 'actual output' compared to the 'expected output'. If the 'actual output' is significantly longer without adding substantial value, penalize the output.",
        "Vague language or contradicting opinions are acceptable as long as they do not introduce factual errors."
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
    model=judger
)

pred_df = pd.read_excel(pred_path)[:100]
testcases=pred_df.apply(lambda row: 
    LLMTestCase(input=row['instruction']+row['input'],actual_output=row['prediction'],expected_output=row['output']),
    axis=1
)
predset = EvaluationDataset(test_cases=testcases)

    
evaluate(predset, [correctness_metric])

✨ You're running DeepEval's latest Correctness (GEval) Metric! (using Custom vllm Server Model, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 100 test case(s) in parallel: |██████████|100% (100/100) [Time Taken: 00:44,  2.23test case/s]




Metrics Summary

  - ❌ Correctness (GEval) (score: 0.3, threshold: 0.5, strict: False, evaluation model: Custom vllm Server Model, reason: 实际输出包含了大部分关键信息，但过于冗长，未遵循摘要的要求。, error: None)

For test case:

  - input: 摘要以下新闻内容。1月19日，英国卫生大臣赛义德·贾维德表示，英格兰地区对奥密克戎毒株的防控工作已取得转机，当地疫情防控进入下一个阶段而并非终结。下周放松防疫限制措施是“我们都可以为之骄傲的时刻”。他强调，政府的行动取得了成效，但同时警告称，这“不是道路的尽头，因为我们还无法根除这种病毒及其未来的变种”（央视）
  - actual output: 英国卫生大臣赛义德·贾维德表示英格兰地区对奥密克戎毒株的防控工作已取得转机，当地疫情防控进入下一个阶段而并非终结。下周放松防疫限制措施是“我们都可以为之骄傲的时刻”。但他也警告称，这“不是道路的尽头，因为我们还无法根除这种病毒及其未来的变种”
  - expected output: 英国卫生大臣：英格兰防疫工作出现转机 但疫情并未终结
  - context: None
  - retrieval context: None


Metrics Summary

  - ❌ Correctness (GEval) (score: 0.2, threshold: 0.5, strict: False, evaluation model: Custom vllm Server Model, reason: Actual output includes all details from expected output but is significantly longer, lacking conciseness., error: None)

For test case:

  - input: 针对以下新闻生成摘要。迈克生物披露定增结果，本次定增募集资金总额为15.74亿元，发行价格为28.11元/股，发行对象最终确定为17家，其中高毅资产获配近2.7亿元。
  - actual output: 迈克

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

EvaluationResult(test_results=[TestResult(name='test_case_6', success=False, metrics_data=[MetricData(name='Correctness (GEval)', threshold=0.5, success=False, score=0.3, reason='实际输出包含了大部分关键信息，但过于冗长，未遵循摘要的要求。', strict_mode=False, evaluation_model='Custom vllm Server Model', error=None, evaluation_cost=None, verbose_logs='Criteria:\nNone \n \nEvaluation Steps:\n[\n    "Check whether the facts in \'actual output\' contradict any facts in \'expected output\'. If there is a contradiction, the output is incorrect.",\n    "Check if the \'actual output\' omits any important details from the \'expected output\'. If details are omitted, the output is less accurate.",\n    "Evaluate the length of the \'actual output\' compared to the \'expected output\'. If the \'actual output\' is significantly longer without adding substantial value, penalize the output.",\n    "Vague language or contradicting opinions are acceptable as long as they do not introduce factual errors."\n]')], conversational=Fals